In [1]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats

dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'

In [21]:
move_level_align = {'FTT': [], 'PTT': []}
move_level_coherence = {'FTT': [], 'PTT': []}

playermove_level_align_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 
playermove_level_coherence_speaker= {'FTT': defaultdict(list), 'PTT': defaultdict(list)}  

for dialogue in listdir(dataset_dir):
    dial_type = dialogue[:3]
    dialogue = path.join(dataset_dir, dialogue)
    dialogue = refexps.load_data(dialogue)
    #move level
    dialogue_move = refexps.move_level(dialogue)
    for move in dialogue_move:
        refs = move[1]
        for obj in refs:
            refobj = refs[obj]
            sim_align = []
            sim_align_speaker = defaultdict(list)
            sim_coherence_speaker = defaultdict(list)
            sim_coherence = []
            for ref in refobj:
                speaker = ref[1]
                sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj))
                if sim != 'undefined':
                    sim_align.append(sim)
                    sim_align_speaker[speaker].append(sim)
            sim_align = np.mean(sim_align)
            comb_ref = combinations(refobj,2)
            for comb in comb_ref:
                if comb[0] != comb[1]:
                    sim = refexps.lex_sim(comb[0], comb[1])
                    if sim != 'undefined':
                        sim_coherence.append(sim)
                        sim_coherence_speaker[comb[0][1]].append(sim)
                        sim_coherence_speaker[comb[1][1]].append(sim)
            sim_coherence = np.mean(sim_coherence)
            #movelevel coherence and alignment (for each object)
            if not np.isnan(sim_coherence):
                move_level_coherence[dial_type].append(sim_coherence)
            if not np.isnan(sim_align):
                move_level_align[dial_type].append(sim_align)
            for s in sim_align_speaker:
                if not np.isnan(np.mean(sim_align_speaker[s])):
                    playermove_level_align_speaker[dial_type][s].append(np.mean(sim_align_speaker[s]))
            for s in sim_coherence_speaker:
                if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                    playermove_level_coherence_speaker[dial_type][s].append(np.mean(sim_coherence_speaker[s]))
    
move_level_align_all = []
move_level_coherence_all = []

playermove_level_align_speaker_all = defaultdict(list)
playermove_level_coherence_speaker_all = defaultdict(list)

results_move = {'FTT': {}, 'PTT': {}, 'ALL': defaultdict(list)}

for dial_type in playermove_level_coherence_speaker:
    move_level_align_all += move_level_align[dial_type]
    move_level_coherence_all += move_level_coherence[dial_type]
    playermove_level_coherence = []
    for s in playermove_level_coherence_speaker[dial_type]:
        playermove_level_coherence += playermove_level_coherence_speaker[dial_type][s]
        playermove_level_coherence_speaker_all[s] += playermove_level_coherence_speaker[dial_type][s]
        results_move[dial_type]['Move coherence '+ s] = playermove_level_coherence_speaker[dial_type][s]
        results_move['ALL']['Move coherence '+ s].extend(playermove_level_coherence_speaker[dial_type][s])
    results_move[dial_type]['Move players coherence'] = playermove_level_coherence
    results_move[dial_type]['Move coherence'] = move_level_coherence[dial_type]
    results_move['ALL']['Move players coherence'].extend(playermove_level_coherence)
    results_move['ALL']['Move coherence'].extend(move_level_coherence[dial_type])
    playermove_level_align = []
    for s in playermove_level_align_speaker[dial_type]:
        playermove_level_align += playermove_level_align_speaker[dial_type][s]
        playermove_level_align_speaker_all[s] += playermove_level_align_speaker[dial_type][s]
        results_move[dial_type]['Move alignment '+ s] = playermove_level_align_speaker[dial_type][s]
        results_move['ALL']['Move alignment '+ s].extend(playermove_level_align_speaker[dial_type][s])
    results_move['ALL']['Move players alignment'].extend(playermove_level_align)
    results_move['ALL']['Move alignment'].extend(move_level_align[dial_type])
    results_move[dial_type]['Move players alignment'] = playermove_level_align
    results_move[dial_type]['Move alignment'] = move_level_align[dial_type]
for d_type in results_move:
    print d_type
    for value in results_move[d_type]:
        dataset = results_move[d_type][value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tVariance: '+ str(np.var(dataset))


ALL
	Move coherence: 
	Mean: 0.206573284874	Variance: 0.0823660647305
	Move alignment e-utts: 
	Mean: 0.0845197184213	Variance: 0.0507075700034
	Move alignment p-utts: 
	Mean: 0.0220938417451	Variance: 0.0145075155714
	Move players coherence: 
	Mean: 0.196688508895	Variance: 0.0810701091939
	Move players alignment: 
	Mean: 0.0417492181765	Variance: 0.0267461146389
	Move coherence p-utts: 
	Mean: 0.201366518535	Variance: 0.0805506046364
	Move coherence e-utts: 
	Mean: 0.188107624933	Variance: 0.0819092645843
	Move alignment: 
	Mean: 0.0242151584408	Variance: 0.00817011681079
PTT
	Move coherence: 
	Mean: 0.234613789225	Variance: 0.0879104234927
	Move alignment e-utts: 
	Mean: 0.0762241344349	Variance: 0.0429660774291
	Move alignment p-utts: 
	Mean: 0.0345850291684	Variance: 0.0227537523922
	Move players coherence: 
	Mean: 0.229024380435	Variance: 0.0873611043256
	Move players alignment: 
	Mean: 0.0478338353895	Variance: 0.0295610842927
	Move coherence p-utts: 
	Mean: 0.236142785157	Varia

In [24]:
values = results_move['FTT'].keys()
print 'DIFFERENCES PTT AND FTT'
for value in values:
    ttest =  stats.ttest_ind(results_move['FTT'][value],results_move['PTT'][value] )
    if ttest[1] < 0.05:
        print value
        print str(ttest)+'\tSignificantly different'
print 'DIFFERENCES E-UTTS AND P-UTTS'
ttest = stats.ttest_ind(results_move['FTT']['Move coherence e-utts'], results_move['FTT']['Move coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['PTT']['Move coherence e-utts'], results_move['PTT']['Move coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['FTT']['Move alignment e-utts'], results_move['FTT']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['PTT']['Move alignment e-utts'], results_move['PTT']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers PTT: '+ str(ttest) + '\tSignificantly different'

DIFFERENCES PTT AND FTT
Move coherence
Ttest_indResult(statistic=-2.2439970521064989, pvalue=0.025160299492474503)	Significantly different
Move alignment p-utts
Ttest_indResult(statistic=-2.5954454003863954, pvalue=0.0096095687315416716)	Significantly different
Move players coherence
Ttest_indResult(statistic=-3.1230839195442726, pvalue=0.0018425709759182993)	Significantly different
Move coherence p-utts
Ttest_indResult(statistic=-2.7256110690370514, pvalue=0.0065969136405423373)	Significantly different
DIFFERENCES E-UTTS AND P-UTTS
Move alignment speakers FTT: Ttest_indResult(statistic=6.3104630637888857, pvalue=4.8777898377212376e-10)	Significantly different
Move alignment speakers PTT: Ttest_indResult(statistic=2.6036750250607978, pvalue=0.0094835564673469933)	Significantly different


In [23]:
game_level_align = {'FTT': [], 'PTT': []}
game_level_coherence = {'FTT': [], 'PTT': []}
playergame_level_align_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 
playergame_level_coherence_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 

for dialogue in listdir(dataset_dir):
    dial_type = dialogue[:3]
    dialogue = refexps.load_data(path.join(dataset_dir, dialogue))
    dialogue_game = refexps.gamerun_level(dialogue)
    for obj in dialogue_game:
        refobj = dialogue_game[obj]
        sim_align = []
        sim_align_speaker = defaultdict(list)
        sim_coherence_speaker = defaultdict(list)
        sim_coherence = []
        for ref in refobj:
            speaker = ref[1]
            sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj))
            if sim != 'undefined':
                sim_align.append(sim)
                sim_align_speaker[speaker].append(sim)
        sim_align = np.mean(sim_align)
        comb_ref = combinations(refobj,2)
        for comb in comb_ref:
            if comb[0] != comb[1]:
                sim = refexps.lex_sim(comb[0], comb[1])
                if sim != 'undefined':
                    sim_coherence.append(sim)
                    sim_coherence_speaker[comb[0][1]].append(sim)
                    sim_coherence_speaker[comb[1][1]].append(sim)
        sim_coherence = np.mean(sim_coherence)
        #gamelevel coherence and alignment (for each object)
        if not np.isnan(sim_coherence) :
            game_level_coherence[dial_type].append(sim_coherence)
        if not np.isnan(sim_align):
            game_level_align[dial_type].append(sim_align)
        for s in sim_align_speaker:
            if not np.isnan(np.mean(sim_align_speaker[s])):
                playergame_level_align_speaker[dial_type][s].append(np.mean(sim_align_speaker[s]))
        for s in sim_coherence_speaker:
            if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                playergame_level_coherence_speaker[dial_type][s].append(np.mean(sim_coherence_speaker[s]))

game_level_align_all = []
game_level_coherence_all = []

playergame_level_align_speaker_all = defaultdict(list)
playergame_level_coherence_speaker_all = defaultdict(list)

results_game = {'FTT': {}, 'PTT': {}, 'ALL': defaultdict(list)}

for dial_type in playergame_level_coherence_speaker:
    game_level_align_all += game_level_align[dial_type]
    game_level_coherence_all += game_level_coherence[dial_type]
    playergame_level_coherence = []
    for s in playergame_level_coherence_speaker[dial_type]:
        playergame_level_coherence += playergame_level_coherence_speaker[dial_type][s]
        playergame_level_coherence_speaker_all[s] += playergame_level_coherence_speaker[dial_type][s]
        results_game[dial_type]['game coherence '+ s] = playergame_level_coherence_speaker[dial_type][s]
        results_game['ALL']['game coherence '+ s].extend(playergame_level_coherence_speaker[dial_type][s])
    results_game[dial_type]['game players coherence'] = playergame_level_coherence
    results_game[dial_type]['game coherence'] = game_level_coherence[dial_type]
    results_game['ALL']['game players coherence'].extend(playergame_level_coherence)
    results_game['ALL']['game coherence'].extend(game_level_coherence[dial_type])
    playergame_level_align = []
    for s in playergame_level_align_speaker[dial_type]:
        playergame_level_align += playergame_level_align_speaker[dial_type][s]
        playergame_level_align_speaker_all[s] += playergame_level_align_speaker[dial_type][s]
        results_game[dial_type]['game alignment '+ s] = playergame_level_align_speaker[dial_type][s]
        results_game['ALL']['game alignment '+ s].extend(playergame_level_align_speaker[dial_type][s])
    results_game['ALL']['game players alignment'].extend(playergame_level_align)
    results_game['ALL']['game alignment'].extend(game_level_align[dial_type])
    results_game[dial_type]['game players alignment'] = playergame_level_align
    results_game[dial_type]['game alignment'] = game_level_align[dial_type]
for d_type in results_game:
    print d_type
    for value in results_game[d_type]:
        dataset = results_game[d_type][value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tVariance: '+ str(np.var(dataset))
print 'GAME RUN LEVEL'
for dial_type in playergame_level_coherence_speaker:
    print dial_type
    game_level_align_all += game_level_align[dial_type]
    game_level_coherence_all += game_level_coherence[dial_type]
    playergame_level_coherence = []
    for s in playergame_level_coherence_speaker[dial_type]:
        playergame_level_coherence += playergame_level_coherence_speaker[dial_type][s]
        playergame_level_coherence_speaker_all[s] += playergame_level_coherence_speaker[dial_type][s]
        print 'Speaker '+ s +' coherence'
        print np.var(playergame_level_coherence_speaker[dial_type][s]), np.mean(playergame_level_coherence_speaker[dial_type][s])
    print 'Game coherence players'
    print np.var(playergame_level_coherence), np.mean(playergame_level_coherence)
    print 'Game coherence'
    print np.var(game_level_coherence[dial_type]), np.mean(game_level_coherence[dial_type])
    playergame_level_align = []
    for s in playergame_level_align_speaker[dial_type]:
        playergame_level_align += playergame_level_align_speaker[dial_type][s]
        playergame_level_align_speaker_all[s] += playergame_level_align_speaker[dial_type][s]
        print 'Speaker '+ s +' alignment'
        print np.var(playergame_level_align_speaker[dial_type][s]), np.mean(playergame_level_align_speaker[dial_type][s])
    print 'Game alignment players'
    print np.var(playergame_level_align), np.mean(playergame_level_align)
    print 'Game alignment'
    print np.var(game_level_align[dial_type]), np.mean(game_level_align[dial_type])


ALL
	game players coherence: 
	Mean: 0.145461311759	Variance: 0.0398325264602
	game coherence e-utts: 
	Mean: 0.131800781514	Variance: 0.0307256759837
	game alignment: 
	Mean: 0.0246849331989	Variance: 0.00509939452707
	game alignment p-utts: 
	Mean: 0.0184436472589	Variance: 0.00608254917159
	game coherence p-utts: 
	Mean: 0.152953612035	Variance: 0.044668816886
	game players alignment: 
	Mean: 0.0403250181937	Variance: 0.016434614997
	game coherence: 
	Mean: 0.15465113007	Variance: 0.0446738924636
	game alignment e-utts: 
	Mean: 0.0870673509991	Variance: 0.0353407576552
PTT
	game players coherence: 
	Mean: 0.178336791372	Variance: 0.0478706552112
	game coherence e-utts: 
	Mean: 0.149552205498	Variance: 0.0332193708746
	game alignment: 
	Mean: 0.0278227692397	Variance: 0.00519542731911
	game alignment p-utts: 
	Mean: 0.0264240270088	Variance: 0.00976238936523
	game coherence p-utts: 
	Mean: 0.193551501049	Variance: 0.0549454689956
	game players alignment: 
	Mean: 0.0411851010542	Varia

In [27]:
values = results_game['FTT'].keys()
print 'DIFFERENCES PTT AND FTT'
for value in values:
    ttest =  stats.ttest_ind(results_game['FTT'][value],results_game['PTT'][value] )
    if ttest[1] < 0.05:
        print value
        print str(ttest)+'\tSignificantly different'
print 'DIFFERENCES E-UTTS AND P-UTTS'
ttest = stats.ttest_ind(results_game['FTT']['game coherence e-utts'], results_game['FTT']['game coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['PTT']['game coherence e-utts'], results_game['PTT']['game coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['FTT']['game alignment e-utts'], results_game['FTT']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['PTT']['game alignment e-utts'], results_game['PTT']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers PTT: '+ str(ttest) + '\tSignificantly different'

DIFFERENCES PTT AND FTT
game players coherence
Ttest_indResult(statistic=-4.1814492536583234, pvalue=3.2919172495152311e-05)	Significantly different
game alignment p-utts
Ttest_indResult(statistic=-2.2542868534695777, pvalue=0.024610185793786356)	Significantly different
game coherence p-utts
Ttest_indResult(statistic=-3.9866566169393773, pvalue=7.8990056406735037e-05)	Significantly different
game coherence
Ttest_indResult(statistic=-3.8051731890530704, pvalue=0.00016240440966135096)	Significantly different
DIFFERENCES E-UTTS AND P-UTTS
Move alignment speakers FTT: Ttest_indResult(statistic=6.1585262553998721, pvalue=1.8786149589784061e-09)	Significantly different
Move alignment speakers PTT: Ttest_indResult(statistic=3.5587730756886904, pvalue=0.00042323415641905407)	Significantly different
